In [ ]:
# Run this if you are using colab
!pip install -qU fsspec==2025.3.0

In [ ]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 peft accelerate bitsandbytes
!pip install -qU json-repair

In [ ]:
from google.colab import userdata
import wandb

hf_token = userdata.get('huggingface')
!huggingface-cli login --token {hf_token}

In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

data_dir = "/content/drive/MyDrive/llm-finetuning"
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

In [ ]:
story = """
هل يمكننا التمييز بين ما إذا كنا نتحدث إلى إنسان آخر أم إلى ذكاء اصطناعي؟ لطالما كان هذا أحد الأسئلة التي يطرحها الناس عند تقييم مدى ذكاء الحواسيب فعلياً.

ويعود أصل هذا التساؤل إلى اختبار تورينغ، الذي وضعه عالم الرياضيات وعلوم الحاسوب الإنجليزي آلان تورينغ عام 1950، محوّلاً التفكير الفلسفي حول ذكاء الآلة إلى اختبار تجريبي للمرة الأولى.

وبحسب هذا الاختبار، إذا كان سلوك الحاسوب غير قابل للتمييز عن سلوك الإنسان، فإنه يُعدّ حينها مظهراً من مظاهر "السلوك الذكي".
لكن عندما قيل إن روبوت محادثة يعمل بالذكاء الاصطناعي قد اجتاز الاختبار لأول مرة في عام 2014 - بدلاً من أن تكون لحظة فاصلة، فقد أثارت جدلاً واسعاً.
لعبة تقليد
واختبار تورينغ هو لعبة تقليد، يتواصل فيها شخص عبر النص مع إنسان آخر وحاسوب.

يُسمح له بطرح أي أسئلة يشاء، قبل أن يُطلب منه في النهاية تحديد أيهما الإنسان وأيهما الآلة.

وقال الدكتور كاميرون جونز، الأستاذ المساعد في علم النفس بجامعة ستوني بروك في نيويورك: "قال تورينغ إنه إذا لم يتمكن الناس من التمييز بشكل موثوق بين البشر والآلات، فلن تكون لدينا أي أسس للقول إن الإنسان قادر على التفكير بينما الآلة غير قادرة على ذلك".

وكان تورينغ قد توقع أنه بحلول عام 2000، ستصبح الحواسيب قادرة على اجتياز هذا الاختبار والتظاهر بأنها بشر، بعد خمس دقائق من الأسئلة، في ما لا يقل عن 30 في المئة من الحالات.
"""

In [ ]:
StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified"
]

EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
    entity_value: str = Field(..., description="The actual name or value of the entity.")
    entity_type: EntityType = Field(..., description="The type of recognized entity.")

class NewsDetails(BaseModel):
    story_title: str = Field(..., min_length=5, max_length=300,
                             description="A fully informative and SEO optimized title of the story.")

    story_keywords: List[str] = Field(..., min_items=1,
                                      description="Relevant keywords associated with the story.")

    story_summary: List[str] = Field(
                                    ..., min_items=1, max_items=5,
                                    description="Summarized key points about the story (1-5 points)."
                                )

    story_category: StoryCategory = Field(..., description="Category of the news story.")

    story_entities: List[Entity] = Field(..., min_items=1, max_items=10,
                                        description="List of identified entities in the story.")


In [ ]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                NewsDetails.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Story Details:",
            "```json"
        ])
    }
]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [ ]:
text = tokenizer.apply_chat_template(
    details_extraction_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)

In [ ]:
data = []
with open("/content/drive/MyDrive/llm-finetuning/dataset/data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
train_sample_sz = 2000

train_ds = data[:train_sample_sz]
eval_ds = data[train_sample_sz:]

os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join("/content/drive/MyDrive/llm-finetuning/dataset", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join("/content/drive/MyDrive/llm-finetuning/dataset", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={
    "train": "/content/drive/MyDrive/llm-finetuning/dataset/train.json",
    "validation": "/content/drive/MyDrive/llm-finetuning/dataset/val.json"
})

def tokenize_function(sample):
    def ensure_string(val):
        if isinstance(val, (dict, list)):
            return json.dumps(val, ensure_ascii=False)
        return str(val)

    task = ensure_string(sample['task'])
    story = ensure_string(sample['story'])
    schema = ensure_string(sample['output_scheme'])
    response = ensure_string(sample['response'])

    messages = [
        {"role": "system", "content": task},
        {"role": "user", "content": f"## Story:\n{story}\n\n## Pydantic Details:\n{schema}\n\n## Story Details:\n```json"},
        {"role": "assistant", "content": response}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=1024,
        padding=False
    )

    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

column_names = dataset["train"].column_names
train_data = dataset["train"].map(tokenize_function, remove_columns=column_names)
val_data = dataset["validation"].map(tokenize_function, remove_columns=column_names)

In [ ]:
print(tokenizer.decode(train_data[0]["input_ids"]))

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./qwen-news-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="paged_adamw_32bit",
    report_to=None,
    remove_unused_columns=True,
    push_to_hub=False
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    pad_to_multiple_of=8,
    return_tensors="pt",
    padding=True
)

trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()